# Welcome to the Modular Multimodal Data Fusion ML Pipeline for stress detection for the WESAD Database

## Table of contents






## Getting Started:
First, download necessary packages, if you are using a venv such as Conda, activate this first.

In [ ]:
%pip install -r requirements.txt

## Data Installation
If you are on Linux, run this cell to download and extract the WESAD dataset automatically, otherwise download manually [here](https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download) and unzip the `WESAD` file into the `wesad` directory i.e. `wesad/WESAD/`

In [ ]:
%cd src/wesad && bash download_database.sh
%cd ../..

## Data Preprocessing

This will automatically extract the biosensor data from the WESAD directory into several merged files in `.pkl` format.

This will take around 20 minutes depending on the machine.

In [ ]:
from src.wesad.data_preprocessing.data_preprocessor import DataPreprocessor

preprocessor = DataPreprocessor()
preprocessor.preprocess()

## Signal Preprocessing Steps
We will preprocess each signal with their respective preprocessing steps:

### Chest Signals

#### ECG
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth band-pass filter of order 3 with cutoff frequencies 0.7 Hz and 3.7 Hz.

#### EMG
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth lowpass filter of order 3 with cutoff frequency 0.5 Hz.

#### EDA
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth lowpass filter of order 2 with cutoff frequency 5 Hz.

#### TEMP
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.

#### RESP
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth band-pass filter of order 3 with cutoff frequencies 0.1 Hz and 0.35 Hz.

#### ACC
- **Smoothing**: Savitzky–Golay filter with window size 31 and order 5.

### Wrist Signals

#### BVP
- **Filtering**: Butterworth band-pass filter of order 3 with cutoff frequencies 0.7 Hz and 3.7 Hz.

#### TEMP
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.

#### ACC
- **Smoothing**: Savitzky–Golay filter with window size 31 and order 5.


In [1]:
from src.wesad.signal_preprocessing import SignalPreprocessor

signal_preprocessor = SignalPreprocessor()
signal_preprocessor.preprocess_signals()

/home/fsociety/Dev/Imperial FYP/ Modular-Multimodal-Stress-Detector/submodules/pyEDA/main.py:126: SyntaxWarning: invalid escape sequence '\p'
  torch.save(model, 'pyEDA\pyEDA\checkpoint.t7')
/home/fsociety/Dev/Imperial FYP/ Modular-Multimodal-Stress-Detector/submodules/pyEDA/main.py:137: SyntaxWarning: invalid escape sequence '\p'
  model = torch.load('pyEDA\pyEDA\checkpoint.t7')


FileNotFoundError: [Errno 2] No such file or directory: 'wesad/WESAD/raw/merged_chest_fltr.pkl'

Now, using the preprocessed `.pkl` files we will make it into a dataloader.

In [ ]:
from wesad.data_preprocessing.dataloader import WesadDataset, WesadPreprocessor, perform_loocv
from torch.utils.data import DataLoader

# Usage example
preprocessor = WesadPreprocessor()
preprocessed_filepath = 'wesad/WESAD/preprocessed_wesad_data.pkl'

# First time processing and saving the data
perform_loocv(preprocessor, use_preprocessed=False, preprocessed_filepath=preprocessed_filepath)

# Later on, loading the preprocessed data for LOOCV
# results = perform_loocv(preprocessor, use_preprocessed=True, preprocessed_filepath=preprocessed_filepath)

In [ ]:
import pandas as pd

df = pd.read_pickle('wesad/WESAD/raw/merged_chest.pkl')

df.head()

In [ ]:
import pandas as pd

# Define file paths
pkl_path1 = 'wesad/WESAD/raw/subj_merged_acc_w.pkl'
pkl_path2 = 'wesad/WESAD/raw/subj_merged_eda_temp_w.pkl'
pkl_path3 = 'wesad/WESAD/raw/subj_merged_bvp_w.pkl'

# Load the data from pickle files
df1 = pd.read_pickle(pkl_path1)
df2 = pd.read_pickle(pkl_path2)
df3 = pd.read_pickle(pkl_path3)

# Merge dataframes on common columns (assumed to be a common index)
# Adjust the merge method and key columns as needed
merged_df = df1.merge(df2, left_index=True, right_index=True, how='inner')
merged_df = merged_df.merge(df3, left_index=True, right_index=True, how='inner')

# Save the merged dataframe to a new pickle file
merged_df.to_pickle('wesad/WESAD/raw/merged_wrist.pkl')
